In [ ]:
%%capture
# Installing everything I need for YouTube tooling + LangChain agent behavior.

%pip install pytube
%pip install youtube-transcript-api==1.1.0
%pip install langchain-community==0.3.16
%pip install langchain==0.3.23
%pip install langchain-openai==0.3.14
%pip install yt-dlp


In [ ]:
import re
import json
import logging
import warnings
from typing import List, Dict

from pytube import YouTube, Search
from youtube_transcript_api import YouTubeTranscriptApi

from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, ToolMessage
from IPython.display import display, JSON

import yt_dlp

# I’m silencing noisy warnings so the notebook output stays readable.
warnings.filterwarnings("ignore")

# Suppress pytube logs
pytube_logger = logging.getLogger("pytube")
pytube_logger.setLevel(logging.ERROR)

# Suppress yt-dlp logs
yt_dpl_logger = logging.getLogger("yt_dlp")
yt_dpl_logger.setLevel(logging.ERROR)


In [ ]:
# I’m using LangChain’s init_chat_model helper so I can plug in a GPT-style model.
# This expects OPENAI_API_KEY to be set in the environment.

from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")


In [ ]:
# IGNORE IF YOU ARE NOT RUNNING LOCALLY
# When I want to run this notebook end-to-end, I’ll set my OpenAI key like this
# (or better: add it in Colab/VSCode environment settings instead of hardcoding it here).

import os
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY_HERE"


In [ ]:
# First custom tool: extract the 11-character video ID from a YouTube URL.

@tool
def extract_video_id(url: str) -> str:
    """
    Extracts the 11-character YouTube video ID from a URL.
    """
    pattern = r"(?:v=|be/|embed/)([a-zA-Z0-9_-]{11})"
    match = re.search(pattern, url)
    return match.group(1) if match else "Error: Invalid YouTube URL"


In [ ]:
print(extract_video_id.name)
print("----------------------------")
print(extract_video_id.description)
print("----------------------------")
print(extract_video_id.func)


In [ ]:
extract_video_id.run("https://www.youtube.com/watch?v=hfIUstzHs9A")


In [ ]:
# I’ll keep all my tools in a single list so I can bind them to the model later.

tools = []
tools.append(extract_video_id)


In [ ]:
# This tool pulls a transcript for a given YouTube video ID.
# Note: transcript availability depends on the video and language.

@tool
def fetch_transcript(video_id: str, language: str = "en") -> str:
    """
    Fetches the transcript of a YouTube video.
    """
    try:
        ytt_api = YouTubeTranscriptApi()
        transcript = ytt_api.fetch(video_id, languages=[language])
        # Joining all text segments into one string
        return " ".join([snippet.text for snippet in transcript.snippets])
    except Exception as e:
        return f"Error: {str(e)}"


In [ ]:
fetch_transcript.run("hfIUstzHs9A")


In [ ]:
tools.append(fetch_transcript)


In [ ]:
# This tool searches YouTube for videos matching a query and returns basic metadata.

from langchain.tools import tool  # This decorator is equivalent to the core one in this context

@tool
def search_youtube(query: str) -> List[Dict[str, str]]:
    """
    Search YouTube for videos matching the query.
    Returns a list of dicts with title, video_id, and URL.
    """
    try:
        s = Search(query)
        return [
            {
                "title": yt.title,
                "video_id": yt.video_id,
                "url": f"https://youtu.be/{yt.video_id}",
            }
            for yt in s.results
        ]
    except Exception as e:
        return f"Error: {str(e)}"


In [ ]:
search_out = search_youtube.run("Generative AI")
display(JSON(search_out))


In [ ]:
tools.append(search_youtube)


In [ ]:
# This tool pulls detailed metadata using yt-dlp.

@tool
def get_full_metadata(url: str) -> dict:
    """
    Extract metadata for a YouTube URL: title, views, duration, channel, likes, comments, chapters.
    """
    with yt_dlp.YoutubeDL({"quiet": True, "logger": yt_dpl_logger}) as ydl:
        info = ydl.extract_info(url, download=False)
        return {
            "title": info.get("title"),
            "views": info.get("view_count"),
            "duration": info.get("duration"),
            "channel": info.get("uploader"),
            "likes": info.get("like_count"),
            "comments": info.get("comment_count"),
            "chapters": info.get("chapters", []),
        }


In [ ]:
meta_data = get_full_metadata.run("https://www.youtube.com/watch?v=T-D1OfcDW1M")
display(JSON(meta_data))


In [ ]:
tools.append(get_full_metadata)


In [ ]:
# This tool collects all available thumbnails for a given YouTube video.

@tool
def get_thumbnails(url: str) -> List[Dict]:
    """
    Get available thumbnails for a YouTube video using its URL.
    """
    try:
        with yt_dlp.YoutubeDL({"quiet": True, "logger": yt_dpl_logger}) as ydl:
            info = ydl.extract_info(url, download=False)

            thumbnails = []
            for t in info.get("thumbnails", []):
                if "url" in t:
                    thumbnails.append({
                        "url": t["url"],
                        "width": t.get("width"),
                        "height": t.get("height"),
                        "resolution": f"{t.get('width', '')}x{t.get('height', '')}".strip("x"),
                    })
            return thumbnails
    except Exception as e:
        return [{"error": f"Failed to get thumbnails: {str(e)}"}]


In [ ]:
thumbnails = get_thumbnails.run("https://www.youtube.com/watch?v=qWHaMrR5WHQ")
display(JSON(thumbnails))


In [ ]:
tools.append(get_thumbnails)


In [ ]:
# Now I bind all of my tools to the LLM so it can call them using tool-calling semantics.

llm_with_tools = llm.bind_tools(tools)


In [ ]:
# I’m inspecting the tool schemas so I can see how the model will perceive them.

for tool in tools:
    schema = {
        "name": tool.name,
        "description": tool.description,
        "parameters": tool.args_schema.schema() if tool.args_schema else {},
        "return": tool.return_type if hasattr(tool, "return_type") else None,
    }
    display(JSON(schema))


In [ ]:
query = "I want to summarize youtube video: https://www.youtube.com/watch?v=T-D1OfcDW1M in english"
print(query)


In [ ]:
messages = [HumanMessage(content=query)]
print(messages)


In [ ]:
response_1 = llm_with_tools.invoke(messages)
response_1


In [ ]:
messages.append(response_1)


In [ ]:
tool_mapping = {
    "get_thumbnails": get_thumbnails,
    "extract_video_id": extract_video_id,
    "fetch_transcript": fetch_transcript,
    "search_youtube": search_youtube,
    "get_full_metadata": get_full_metadata,
}


In [ ]:
tool_calls_1 = response_1.tool_calls
display(JSON(tool_calls_1))


In [ ]:
tool_name = tool_calls_1[0]["name"]
print(tool_name)

tool_call_id = tool_calls_1[0]["id"]
print(tool_call_id)

args = tool_calls_1[0]["args"]
print(args)


In [ ]:
my_tool = tool_mapping[tool_calls_1[0]["name"]]
video_id = my_tool.invoke(tool_calls_1[0]["args"])
video_id


In [ ]:
messages.append(
    ToolMessage(
        content=video_id,
        tool_call_id=tool_calls_1[0]["id"],
    )
)


In [ ]:
response_2 = llm_with_tools.invoke(messages)
response_2


In [ ]:
messages.append(response_2)


In [ ]:
tool_calls_2 = response_2.tool_calls
tool_calls_2


In [ ]:
fetch_transcript_tool_output = tool_mapping[tool_calls_2[0]["name"]].invoke(
    tool_calls_2[0]["args"]
)
fetch_transcript_tool_output


In [ ]:
messages.append(
    ToolMessage(
        content=fetch_transcript_tool_output,
        tool_call_id=tool_calls_2[0]["id"],
    )
)


In [ ]:
summary = llm_with_tools.invoke(messages)
summary


In [ ]:
# Reusable helper for executing a tool call and wrapping it as a ToolMessage.

def execute_tool(tool_call):
    """Execute single tool call and return ToolMessage."""
    try:
        result = tool_mapping[tool_call["name"]].invoke(tool_call["args"])
        return ToolMessage(
            content=str(result),
            tool_call_id=tool_call["id"],
        )
    except Exception as e:
        return ToolMessage(
            content=f"Error: {str(e)}",
            tool_call_id=tool_call["id"],
        )


In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda


In [ ]:
# Here I’m building a small chain that:
# 1) Gets the URL
# 2) Lets the LLM choose tools step by step
# 3) Executes them
# 4) Produces the final summary

summarization_chain = (
    RunnablePassthrough.assign(
        messages=lambda x: [HumanMessage(content=x["query"])]
    )
    | RunnablePassthrough.assign(
        ai_response=lambda x: llm_with_tools.invoke(x["messages"])
    )
    | RunnablePassthrough.assign(
        tool_messages=lambda x: [
            execute_tool(tc) for tc in x["ai_response"].tool_calls
        ]
    )
    | RunnablePassthrough.assign(
        messages=lambda x: x["messages"] + [x["ai_response"]] + x["tool_messages"]
    )
    | RunnablePassthrough.assign(
        ai_response2=lambda x: llm_with_tools.invoke(x["messages"])
    )
    | RunnablePassthrough.assign(
        tool_messages2=lambda x: [
            execute_tool(tc) for tc in x["ai_response2"].tool_calls
        ]
    )
    | RunnablePassthrough.assign(
        messages=lambda x: x["messages"] + [x["ai_response2"]] + x["tool_messages2"]
    )
    | RunnablePassthrough.assign(
        summary=lambda x: llm_with_tools.invoke(x["messages"]).content
    )
    | RunnableLambda(lambda x: x["summary"])
)


In [ ]:
result = summarization_chain.invoke({
    "query": "Summarize this YouTube video: https://www.youtube.com/watch?v=1bUy-1hGZpI"
})

print("Video Summary:\n", result)


In [ ]:
initial_setup = RunnablePassthrough.assign(
    messages=lambda x: [HumanMessage(content=x["query"])]
)

first_llm_call = RunnablePassthrough.assign(
    ai_response=lambda x: llm_with_tools.invoke(x["messages"])
)

first_tool_processing = RunnablePassthrough.assign(
    tool_messages=lambda x: [
        execute_tool(tc) for tc in x["ai_response"].tool_calls
    ]
).assign(
    messages=lambda x: x["messages"] + [x["ai_response"]] + x["tool_messages"]
)

second_llm_call = RunnablePassthrough.assign(
    ai_response2=lambda x: llm_with_tools.invoke(x["messages"])
)

second_tool_processing = RunnablePassthrough.assign(
    tool_messages2=lambda x: [
        execute_tool(tc) for tc in x["ai_response2"].tool_calls
    ]
).assign(
    messages=lambda x: x["messages"] + [x["ai_response2"]] + x["tool_messages2"]
)


In [ ]:
final_summary = (
    RunnablePassthrough.assign(
        summary=lambda x: llm_with_tools.invoke(x["messages"]).content
    )
    | RunnableLambda(lambda x: x["summary"])
)


In [ ]:
chain = (
    initial_setup
    | first_llm_call
    | first_tool_processing
    | second_llm_call
    | second_tool_processing
    | final_summary
)


In [ ]:
query = {
    "query": "I want to summarize youtube video: https://www.youtube.com/watch?v=T-D1OfcDW1M in english"
}
result = summarization_chain.invoke(query)
print("Video Summary:\n", result)


In [ ]:
query = {"query": "Get top 3 youtube videos in India and their metadata"}
try:
    result = summarization_chain.invoke(query)
    print("Video Summary:\n", result)
except Exception as e:
    print("Non-critical network error:", e)


In [ ]:
from langchain_core.runnables import RunnableBranch, RunnableLambda
from langchain_core.messages import HumanMessage, ToolMessage

def execute_tool(tool_call):
    """Execute single tool call and return ToolMessage."""
    try:
        result = tool_mapping[tool_call["name"]].invoke(tool_call["args"])
        content = json.dumps(result) if isinstance(result, (dict, list)) else str(result)
    except Exception as e:
        content = f"Error: {str(e)}"

    return ToolMessage(
        content=content,
        tool_call_id=tool_call["id"],
    )


In [ ]:
def process_tool_calls(messages):
    """Run tools for the last message and ask the LLM what to do next."""
    last_message = messages[-1]

    tool_messages = [
        execute_tool(tc)
        for tc in getattr(last_message, "tool_calls", [])
    ]

    updated_messages = messages + tool_messages
    next_ai_response = llm_with_tools.invoke(updated_messages)

    return updated_messages + [next_ai_response]


In [ ]:
def should_continue(messages):
    """Check if the latest message still contains pending tool calls."""
    last_message = messages[-1]
    return bool(getattr(last_message, "tool_calls", None))


In [ ]:
def _recursive_chain(messages):
    """Recursively process tool calls until the LLM no longer requests tools."""
    if should_continue(messages):
        new_messages = process_tool_calls(messages)
        return _recursive_chain(new_messages)
    return messages

recursive_chain = RunnableLambda(_recursive_chain)


In [ ]:
universal_chain = (
    RunnableLambda(lambda x: [HumanMessage(content=x["query"])])
    | RunnableLambda(lambda messages: messages + [llm_with_tools.invoke(messages)])
    | recursive_chain
)


In [ ]:
query_us = {"query": "Show top 3 US trending videos with metadata and thumbnails"}

try:
    response = universal_chain.invoke(query_us)
    print("\nUS Trending Videos:\n", response[-1])
except Exception as e:
    print("Non-critical network error while fetching US trending videos:", e)
